## 🧠 Redefinición del enfoque: Dataset sintético con estrategia mixta

### 🔍 Problema detectado

En la primera versión del dataset, tanto el `tipo_de_semana` como el `burnout_index` eran **totalmente deterministas**, es decir, podían deducirse directamente a partir de reglas fijas sobre las 10 variables.

➡️ Esto hacía **inviable aplicar técnicas de Machine Learning supervisado**, ya que el modelo no tenía nada que aprender: solo replicaba reglas.

---

### 🛠️ Solución propuesta: estrategia mixta de generación

Adoptamos un enfoque **probabilístico y controlado** para generar las etiquetas (`tipo_de_semana` y `burnout_index`) a partir de combinaciones ambiguas y semi-aleatorias de features.

---

### 📊 Etiquetado del tipo de semana (`week_type`)

Producimos tres tipos de registros:

| Tipo de generación         | % sugerido | Descripción                                                                 |
|---------------------------|------------|-----------------------------------------------------------------------------|
| Semanas “típicas” (5+5)   | 60%        | 5 variables coherentes con un tipo de semana + 5 mezcladas aleatoriamente |
| Semanas “ambiguas” (10r)  | 30%        | Cada variable viene de un tipo distinto de semana                         |
| Semanas “coherentes full” | 10%        | Todas las variables coherentes con un único tipo (casos extremos)         |

➡️ La clase final (`week_type`) se define con `random.choices()` en función de la proporción de variables por tipo → esto **rompe el determinismo**.

---

### 📈 Etiquetado del burnout index (`burnout_index`)

También aplicamos una estrategia mixta:

| Modo de generación            | % sugerido | Descripción                                     |
|------------------------------|------------|-------------------------------------------------|
| Determinístico puro          | 10%        | Valor fijo dentro del rango definido por clase |
| Fórmula con ruido            | 60%        | Promedio de los features normalizados + ruido  |
| Modelo aleatorio controlado  | 30%        | Valor aleatorio dentro del rango de la clase   |

➡️ Esto permite que `burnout_index` **no sea una función exacta del tipo de semana**, sino una variable continua con cierta dispersión y ruido controlado.

---

### 🤖 ¿Qué técnicas de ML se podrán aplicar ahora?

Con este enfoque, logramos un único dataset con dos objetivos predictivos viables para tareas de aprendizaje supervisado:

| Objetivo                    | Target           | Técnica de ML supervisado |
|----------------------------|------------------|---------------------------|
| Clasificar tipo de semana  | `week_type`      | Clasificación             |
| Estimar nivel de burnout   | `burnout_index`  | Regresión                 |

---

### ✅ Resultado esperado

Cuando ingrese un nuevo registro real con sus 10 variables, el sistema podrá:

- Predecir su **tipo de semana** con un modelo de clasificación
- Estimar su **índice de burnout** con un modelo de regresión

---

### 🤖 Algoritmos de ML aplicables

#### 📊 Modelos de Clasificación (para `week_type`)
Podemos aplicar cualquiera de los siguientes modelos vistos en el bootcamp:

- `LogisticRegression`
- `KNeighborsClassifier`
- `RandomForestClassifier`
- `GradientBoostingClassifier`
- `XGBClassifier` (si tenés XGBoost instalado)

#### 📈 Modelos de Regresión (para `burnout_index`)
Estos modelos permiten predecir el burnout index como valor continuo:

- `LinearRegression`
- `Ridge` / `Lasso` (con regularización)
- `KNeighborsRegressor`
- `RandomForestRegressor`
- `GradientBoostingRegressor`
- `XGBRegressor`

🧪 Todos estos modelos pueden ser combinados con `GridSearchCV`, `StandardScaler`, `Pipeline` y otras técnicas vistas en clase.
---

### 📌 Ventajas del nuevo enfoque

- Representa la ambigüedad real de los datos humanos
- Permite entrenar y evaluar modelos de ML de forma realista
- Conserva la lógica de los tramos definidos, pero **con ruido y mezcla**
- Abre la puerta a tareas de mejora de performance, visualización, explicación de modelos y más


## 🔥 Estrategia mixta para generar el `burnout_index`

Una vez determinado el tipo de semana (`week_type`), se conoce el **rango válido** en el que debe caer el `burnout_index`:

| Tipo de semana (`week_type`) | Rango del `burnout_index` |
|------------------------------|----------------------------|
| 1 – Semana saludable         | 1.0 – 3.0                  |
| 2 – Carga aceptable          | 3.1 – 5.0                  |
| 3 – Exceso de carga          | 5.1 – 7.5                  |
| 4 – Agotamiento extremo      | 7.6 – 10.0                 |

A partir de ese rango, aplicamos una **estrategia mixta** para generar el valor del burnout index. Esta estrategia combina tres métodos:

---

### 🧠 1. Determinístico puro (10%)

- Se normalizan los 10 features (valores entre 0 y 1).
- Se aplica una fórmula lineal ponderada (por ejemplo):

```python
score = w1 * f1 + w2 * f2 + ... + w10 * f10
```

- Luego se reescala al rango correspondiente según `week_type`.
- Finalmente, se aplica:

```python
burnout = np.clip(score, min_rango, max_rango)
```

---

### 🌪️ 2. Fórmula con ruido (60%)

- Igual que el determinístico, pero se suma **ruido gaussiano** para introducir variabilidad:

```python
burnout = score + np.random.normal(0, 0.3)
burnout = np.clip(burnout, min_rango, max_rango)
```

Esto permite que semanas similares puedan tener índices diferentes, manteniéndose dentro del rango.

---

### 🎲 3. Modelo aleatorio controlado (30%)

- Se genera un valor completamente aleatorio dentro del rango permitido por tipo de semana:

```python
burnout = np.random.uniform(min_rango, max_rango)
```

Esto introduce diversidad y evita que el modelo aprenda relaciones triviales entre features y burnout index.

---

### 🧩 ¿Qué aporta esta estrategia?

- El enfoque **determinístico puro** garantiza ejemplos sólidos y coherentes.
- El enfoque **con ruido** simula variabilidad humana razonable.
- El enfoque **aleatorio controlado** genera casos ambiguos que ayudan a evitar el sobreajuste.

---

### ✅ Aplicación de ML

Con esta estrategia, se genera un dataset sintético que permite aplicar **ML supervisado de regresión** con algoritmos como:

- `LinearRegression`
- `Ridge`, `Lasso`
- `RandomForestRegressor`
- `GradientBoosting`
- `XGBoost`, `CatBoost`

El objetivo es que el modelo aprenda a estimar un `burnout_index` continuo, no determinista, en base a los 10 features semanales.


## 🧠 ¿Cómo se aplica el modelo de regresión para predecir el `burnout_index`?

Una vez que tenemos los 10 features de una persona (extraídos de su semana digital), el flujo de predicción es el siguiente:

---

### 🔢 Paso 1. Predecir el tipo de semana (`week_type`)

Se utiliza un modelo de **Machine Learning supervisado de clasificación**, entrenado previamente, que predice el tipo de semana (`week_type`) en base a las 10 variables.

---

### 📈 Paso 2. Predecir el `burnout_index`

Se utiliza un **modelo de regresión supervisado** para estimar el índice de burnout como valor continuo entre 1.0 y 10.0.

❗️ Importante: **no se entrena un modelo distinto por cada tipo de semana**. En cambio, se entrena **un único modelo de regresión**, usando todos los registros del dataset.

---

### ✅ ¿Por qué un solo modelo?

- Permite aprovechar **todos los datos disponibles**.
- El modelo puede **aprender patrones comunes entre tipos de semana**.
- Mejora la **capacidad de generalización**.

---

### 🧩 ¿Qué inputs usa el modelo?

El modelo de regresión puede entrenarse con:

- Las 10 variables de la semana (`f1` a `f10`).
- Opcionalmente, el `week_type` predicho, codificado como variable categórica (ej. one-hot).

---

### 🔒 ¿Cómo asegurar que el valor quede en el rango válido?

Al final, podemos aplicar un `clip()` para que el resultado quede dentro del rango esperado según el tipo de semana predicho:

```python
burnout_index = np.clip(predicted_score, min_range, max_range)
```

Esto mantiene la coherencia con la lógica de generación del dataset sintético.

---

En resumen:

| Etapa              | Modelo               | Tipo de ML        | Target          |
|--------------------|----------------------|--------------------|------------------|
| Clasificación      | week_type            | Clasificación      | Tipo de semana (1 a 4) |
| Predicción continua| burnout_index        | Regresión supervisada | Índice de burnout (1.0 a 10.0) |

In [35]:
import pandas as pd

# Carga del dataset generado
df = pd.read_csv("raw_data/synthetic_burnout_dataset.csv")

# Visualizamos las primeras filas
df.head(30)

,num_events,num_events_outside_hours,total_meeting_hours,avg_meeting_duration,meetings_weekend,emails_sent,emails_sent_out_of_hours,docs_created,docs_edited,num_meetings_no_breaks,emails_received,num_overlapping_meetings,tipo_de_semana,burnout_index
0,35.45,0.43,2.16,99.24,4.95,19.89,0.54,0.54,15.59,0.00,13.99,0.00,semana_saludable,2.44
1,39.04,9.46,18.05,118.69,3.83,37.50,12.96,3.20,15.81,6.08,8.59,7.13,semana_saludable,2.78
2,13.09,2.80,20.34,58.45,1.00,25.62,14.36,3.61,6.12,1.69,30.74,5.47,semana_carga_aceptable,3.74
3,13.62,2.74,12.15,57.97,0.00,41.16,3.02,3.24,6.05,1.84,29.96,1.00,semana_carga_aceptable,4.88
4,25.76,6.29,34.02,73.06,0.00,28.44,4.50,3.51,17.98,8.78,8.14,5.24,semana_agotamiento_extremo,8.83
5,11.11,0.38,25.76,58.78,1.00,11.29,4.36,3.55,6.88,7.03,26.99,0.00,semana_carga_aceptable,3.67
6,7.83,2.67,15.59,24.72,0.00,40.46,1.86,1.71,0.90,0.00,46.82,0.00,semana_carga_excesiva,5.49
7,39.72,2.84,27.82,96.52,3.64,32.53,0.54,11.31,19.04,6.88,68.69,2.92,semana_agotamiento_extremo,9.03
8,36.32,2.86,33.13,25.02,3.66,42.08,9.49,3.83,0.02,6.41,79.17,0.00,semana_agotamiento_extremo,8.71
9,23.34,2.68,13.71,49.25,1.00,14.03,6.40,3.06,4.95,1.99,6.80,1.00,semana_carga_aceptable,3.55


In [36]:
df.shape  # cantidad de filas y columnas

df['tipo_de_semana'].value_counts()  # distribución por tipo

tipo_de_semana
semana_carga_excesiva         275
semana_saludable              258
semana_carga_aceptable        254
semana_agotamiento_extremo    213
Name: count, dtype: int64

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   num_events                1000 non-null   float64
 1   num_events_outside_hours  1000 non-null   float64
 2   total_meeting_hours       1000 non-null   float64
 3   avg_meeting_duration      1000 non-null   float64
 4   meetings_weekend          1000 non-null   float64
 5   emails_sent               1000 non-null   float64
 6   emails_sent_out_of_hours  1000 non-null   float64
 7   docs_created              1000 non-null   float64
 8   docs_edited               1000 non-null   float64
 9   num_meetings_no_breaks    1000 non-null   float64
 10  emails_received           1000 non-null   float64
 11  num_overlapping_meetings  1000 non-null   float64
 12  tipo_de_semana            1000 non-null   object 
 13  burnout_index             1000 non-null   float64
dtypes: float6

In [38]:
df.tail(200)

,num_events,num_events_outside_hours,total_meeting_hours,avg_meeting_duration,meetings_weekend,emails_sent,emails_sent_out_of_hours,docs_created,docs_edited,num_meetings_no_breaks,emails_received,num_overlapping_meetings,tipo_de_semana,burnout_index
800,12.19,4.08,20.45,66.14,0.00,44.83,13.73,1.12,5.17,8.29,65.32,6.69,semana_saludable,1.33
801,13.31,4.54,11.07,62.01,0.00,19.87,0.92,3.93,0.52,4.45,32.25,2.52,semana_carga_aceptable,3.47
802,39.68,4.92,11.68,73.49,2.00,29.03,1.37,8.63,13.37,9.44,30.14,1.00,semana_agotamiento_extremo,8.54
803,3.27,0.97,10.38,9.74,0.00,33.74,12.31,3.79,0.05,1.30,10.32,1.00,semana_carga_aceptable,3.35
804,15.32,4.07,1.61,53.13,0.00,11.31,7.91,3.35,10.35,3.48,53.20,1.00,semana_carga_excesiva,6.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,23.16,4.85,22.13,69.81,2.00,28.21,7.37,7.87,8.98,3.15,48.09,2.66,semana_carga_excesiva,6.50
996,34.50,8.01,29.85,80.89,4.02,32.11,12.80,11.08,19.60,9.16,59.09,8.77,semana_agotamiento_extremo,9.65
997,39.94,6.13,29.53,110.35,3.71,38.61,10.02,13.77,16.84,6.01,74.18,8.99,semana_agotamiento_extremo,9.69
998,35.58,8.46,33.77,91.40,4.56,48.92,14.49,11.12,16.39,7.01,61.41,6.93,semana_agotamiento_extremo,9.86
